In [ ]:
import random
from ME_BPER import ME_BPER
#from ME_IPER import ME_IPER
from ME_LOC import ME_LOC
from ME_ORG import ME_ORG

from scripts.load_data import mapping, read_tsv_file, extract_labeled_tokens

ModuleNotFoundError: No module named 'nlp25_project_gr2'

In [4]:
def collect_entity_strings(data, target_label_prefix):
    """
    Collects labeled spans as strings from the dataset. Multi-token spans are joined with spaces.
    
    Args:
        data (List[Dict]): Dataset containing 'tokens' and 'tags' for each sentence.
        target_label_prefix (str): Label prefix to filter for (e.g., 'B-LOC', 'B-ORG').
        
    Returns:
        Set[str]: A set of labeled token strings (e.g., {'Beirut', 'Al Mawsil al Jadidah'})
    """
    grouped_strings = set()

    for item in data:
        tokens = item['tokens']
        tags = item['tags']

        i = 0
        while i < len(tokens):
            tag = tags[i]

            if tag.startswith(target_label_prefix):
                span_tokens = [tokens[i]]
                i += 1
                while i < len(tokens) and tags[i].startswith('I'):
                    span_tokens.append(tokens[i])
                    i += 1

                # Join tokens into a single string and add to the set
                entity_string = ' '.join(span_tokens)
                grouped_strings.add(entity_string)
            else:
                i += 1

    return grouped_strings


In [8]:
# overlap between train, dev, test and MENAPT NEs
ME_LOC_tokens = collect_entity_strings(ME_LOC, target_label_prefix = "B-LOC")

ME_ORG_tokens = collect_entity_strings(ME_ORG, target_label_prefix = "B-ORG")

print(ME_LOC_tokens)
print(len(ME_LOC_tokens))
print(len(ME_LOC))

print("\n")

print(ME_ORG_tokens)
print(len(ME_ORG_tokens))
print(len(ME_ORG))


{'Al Ain', 'Al Minya', 'Kufa', 'Kermanshah', 'Khoy', 'Gorgan', 'Jidda', 'Birjand', 'Qurayyat', 'Corlu', 'Chalus', 'Orumiyeh', 'Al Kut', 'Yozgat', 'Kafr ash Shaykh', 'Al Ahmadi', 'Mashhad', 'Nazilli', 'Zanjan', 'Dubai', 'Khan Yunis', 'Dehdasht', 'Zonguldak', 'Kuşadası', 'Mahabad', 'Housh Eissa', 'Qina', 'Çubuk', 'Oman', 'As Samawah', 'Torbat-e Heydariyeh', 'Çerkezköy', 'Taiz', 'Daraa', 'Luxor', 'Sari', 'Amran', 'Faqus', 'İnegöl', 'Kızıltepe', 'Nevşehir', 'Saveh', 'Egypten', 'Najran', "Ha'il", 'Hakkari', 'Deir ez-Zor', 'Tripoli', 'Sakakah', 'Iskenderun', 'Manisa', 'Denizli', 'Rustaq', 'Tyrkiet', 'Üsküdarr', 'Latakia', 'Sabzevar', 'Osmaniye', 'Zefta', 'Ismailia', 'Tarsus', 'Kuwait', 'Rafah', 'Al Harithah', 'Patnos', 'Jamjamal', 'Yanbu', 'As-Salt', 'Edfu', 'Toukh', 'Robat Karim', 'Langarud', 'Sirjan', 'Dahuk', 'Nahavand', 'Al Bab', 'Al Fujayrah', 'Al-Arish', 'Samalut', 'Al Muharraq', 'Marand', 'Abu Dhabi', 'Ashmun', 'Izmir', 'Darab', 'Kairo', 'Esenyurt', 'Zarqa', 'Adapazari', 'Aksaray', 'I

In [ ]:
train_tokens = 
dev_tokens = 
test_tokens = 

In [ ]:
def data_aug_replace(dataset, sentence_amount):
    # First, filter sentences that have at least one non-"O" tag
    eligible_sentences = [sent for sent in dataset if any(tag != "O" for tag in sent["ner_tags"])]

    # Select up to sentence_amount randomly from the eligible ones
    selected_sentences = random.sample(eligible_sentences, min(sentence_amount, len(eligible_sentences)))


    for sent in selected_sentences:
        i = 0

        while i<len(sent["tokens"]):
            tag = sent["ner_tags"][i]

            if tag == 'B-PER':
                replace = random.choice(ME_BPER)
                sent["ner_tags"][i] = replace
                ME_BPER.remove(replace)
                i+=1

            elif tag == 'I-PER':
                replace = random.choice(ME_IPER)
                sent["ner_tags"][i] = replace
                ME_IPER.remove(replace)
                i+=1

            elif tag == 'B-LOC':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-LOC":
                    span_len += 1
                    i += 1

                same_length_LOC = []
                for loc in ME_LOC:
                    if len(loc) == span_len:
                        same_length_LOC.append(loc)
                if same_length_LOC:
                    replace = random.choice(same_length_LOC)
                    sent["tokens"][span_start:span_start+span_len] = replace
                    ME_LOC.remove(replace)
            
            elif tag == 'B-ORG':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-ORG":
                    span_len += 1
                    i += 1

                same_length_ORG = []
                for org in ME_ORG:
                    if len(org) == span_len:
                        same_length_ORG.append(org)
                if same_length_ORG:
                    replace = random.choice(same_length_ORG)
                    sent["tokens"][span_start:span_start+span_len] = replace
                    ME_ORG.remove(replace)

            else:
                i+=1
